In [16]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
import spacy
import re
import nltk
from nltk import WordNetLemmatizer
from langdetect import detect


In [3]:
news = pd.read_csv("C:/Users/jchreim/Desktop/Fundementals of NLP/Group_assignment/fake_real_news.csv")
news.head()

,title,text,idd,label
0,‘A target on Roe v. Wade ’: Oklahoma bill maki...,UPDATE: Gov. Fallin vetoed the bill on Friday....,Fq+C96tcx+,REAL
1,Study: women had to drive 4 times farther afte...,Ever since Texas laws closed about half of the...,bHUqK!pgmv,REAL
2,"Trump, Clinton clash in dueling DC speeches","Donald Trump and Hillary Clinton, now at the s...",4Y4Ubf%aTi,REAL
3,Grand jury in Texas indicts activists behind P...,A Houston grand jury investigating criminal al...,_CoY89SJ@K,REAL
4,"As Reproductive Rights Hang In The Balance, De...",WASHINGTON -- Forty-three years after the Supr...,+rJHoRQVLe,REAL


In [13]:
news['description'] = news['title']+news['text']
news.drop(['title','text'], axis=1, inplace=True)
news.head()

,idd,label,description
0,Fq+C96tcx+,REAL,‘A target on Roe v. Wade ’: Oklahoma bill maki...
1,bHUqK!pgmv,REAL,Study: women had to drive 4 times farther afte...
2,4Y4Ubf%aTi,REAL,"Trump, Clinton clash in dueling DC speechesDon..."
3,_CoY89SJ@K,REAL,Grand jury in Texas indicts activists behind P...
4,+rJHoRQVLe,REAL,"As Reproductive Rights Hang In The Balance, De..."


## Checking Languages

In [31]:
news["text_final"]=news["description"].astype(str)

In [32]:
languages = []

# Loop over the sentences in the list and detect their language
for sentence in news["text_final"]:
    languages.append(detect(sentence))
    
# Assign the list to a new feature 
news['language'] = languages


In [24]:
news.shape

(4594, 4)

Keep the English articles

In [33]:
news=news[news['language']=='en']
news.shape

(4591, 5)

## Data Cleaning

In [27]:
import nltk
from nltk.corpus import opinion_lexicon
from nltk import word_tokenize 

def Positive_words(text):


    nltk.download('opinion_lexicon')

    pos_list=set(opinion_lexicon.positive())

    text = word_tokenize(text)

    return len([token for token in text if token in pos_list])

In [28]:
# Function that counts number of negative words in a sentence
def Negative_words(text):


    nltk.download('opinion_lexicon')

    neg_list=set(opinion_lexicon.negative())

    text = word_tokenize(text)

    return len([token for token in text if token in neg_list])


In [34]:
def generateVariables (df):

    nbwords=[]
    exclamations=[]
    Pos_count=[]
    Neg_count=[]
  
    for sentence in news['text_final'] :
      nbwords.append(len(sentence.split()))
      exclamations.append(sentence.count('!'))
      Pos_count.append(Positive_words(sentence))
      Neg_count.append(Negative_words(sentence))

    df['WordCount']=nbwords
    df['ExclCount']=exclamations
    df['PosCount']=Pos_count
    df['NegCount']=Neg_count

    return df

In [35]:
news=generateVariables(news)
news.head()

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\jchreim\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\jchreim\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\jchreim\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\jchreim\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\jchreim\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\jchreim\AppData\Roaming\nltk_data...
[nltk_data]   Pac

,idd,label,description,language,text_final,WordCount,ExclCount,PosCount,NegCount
0,Fq+C96tcx+,REAL,‘A target on Roe v. Wade ’: Oklahoma bill maki...,en,‘A target on Roe v. Wade ’: Oklahoma bill maki...,1107,0,14,28
1,bHUqK!pgmv,REAL,Study: women had to drive 4 times farther afte...,en,Study: women had to drive 4 times farther afte...,705,0,14,11
2,4Y4Ubf%aTi,REAL,"Trump, Clinton clash in dueling DC speechesDon...",en,"Trump, Clinton clash in dueling DC speechesDon...",526,2,17,21
3,_CoY89SJ@K,REAL,Grand jury in Texas indicts activists behind P...,en,Grand jury in Texas indicts activists behind P...,822,0,19,25
4,+rJHoRQVLe,REAL,"As Reproductive Rights Hang In The Balance, De...",en,"As Reproductive Rights Hang In The Balance, De...",441,0,7,12


In [15]:
from nltk.tokenize import word_tokenize
import nltk
import spacy
nltk.download('stopwords')
nlp = spacy.load('en_core_web_sm')
nlp.disable_pipes('ner',"tagger","parser")
stop_words = nltk.corpus.stopwords.words('english') # initialize the stop word list

def normalize_document(doc):
  #Remove non necessary characters
  text = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
  #Tokenize
  text = word_tokenize(text)
  #Remove numeric 
  text = [token for token in text if not token.isnumeric()]
  # Remove stop words
  stop_words = nltk.corpus.stopwords.words('english') # initialize the stop word list
  text = [token for token in text if token not in stop_words]

  # Lemmatize
  # Instantiate the WordNetLemmatizer
  wordnet_lemmatizer = WordNetLemmatizer()

  # Lemmatize all tokens into a new list: lemmatized
  text_lemmatized1 = [wordnet_lemmatizer.lemmatize(t) for t in text]

  return text_lemmatized1

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jchreim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
from itertools import chain

norm_description= [normalize_document(r) for r in news['description']]

# Create a list here of all the words (based on unnorm_reviews and norm_reviews created above)
news['description'] = list(chain.from_iterable(norm_description))  # Flatten list of lists

ValueError: Length of values (2316454) does not match length of index (4591)